<a href="https://colab.research.google.com/github/JJang770/GitHub_tutorial/blob/main/Torchvision_tr01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('./drive')

In [ ]:
!pip3 install torchinfo

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

from torchinfo import summary

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

print(f'Pytorch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')

In [ ]:
IMG_SIZE = 480
THRESHOLD = 0.95

https://pytorch.org/docs/stable/torchvision/models.html

In [ ]:
model = models.detection.keypointrcnn_resnet50_fpn(pretrained=True).eval()

In [ ]:
summary(model)

In [ ]:
img = Image.open('/content/drive/MyDrive/fastcampus_vision/imgs/07.jpg')
img = img.resize((IMG_SIZE,int(img.height*IMG_SIZE/img.width)))

plt.figure(figsize=(16,16))
plt.imshow(img)

In [ ]:
trf = transforms.Compose([transforms.ToTensor()])

In [ ]:
input_img = trf(img)
print(input_img.shape)
input = input_img.unsqueeze(0)
print(input.shape)

In [ ]:
out = model(input)[0]

In [ ]:
print(out)

In [ ]:
print(f'out type: {type(out)}')
print(f'out keys: {out.keys()}')

In [ ]:
box_arr = out['boxes']
N = box_arr.shape[0]
print(f'boxes shape: {box_arr.shape}')
print(f'N :{N}')

# Inference

- boxes (Tensor[N, 4]): the ground-truth boxes in [x0, y0, x1, y1] format, with values between 0 and H and 0 and W
- labels (Tensor[N]): the class label for each ground-truth box
- keypoints (Tensor[N, K, 3]): the K keypoints location for each of the N instances, in the format [x, y, visibility], where visibility=0 means that the keypoint is not visible.

In [ ]:
codes = [
    Path.MOVETO,
    Path.LINETO,
    Path.LINETO
]

fig, ax = plt.subplots(1,figsize=(16,16))
ax.imshow(img)

for box, score, keypoints in zip(out['boxes'], out['scores'], out['keypoints']):
  score = score.detach().numpy()
  print(f'score : {score:.5f}')
  if score < THRESHOLD:
    continue

  box = box.detach().numpy()
  keypoints = keypoints.detach().numpy()[:,:2]
  print(box[:4])
  rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=2, edgecolor='b', facecolor='none')
  ax.add_patch(rect)

  #17 keypoints
  for k in keypoints:
    circle = patches.Circle((k[0],k[1]),radius=2, facecolor='r')
    ax.add_patch(circle)

  #draw path
  #left arm
  path = Path(keypoints[5:10:2], codes)
  line = patches.PathPatch(path, linewidth=2, facecolor='none', edgecolor='r')
  ax.add_patch(line)

  #right arm
  path = Path(keypoints[6:11:2],codes)
  line = patches.PathPatch(path, linewidth=2, facecolor='none', edgecolor='r')
  ax.add_patch(line)

  #left leg
  path = Path(keypoints[11:16:2],codes)
  line = patches.PathPatch(path, linewidth=2, facecolor='none', edgecolor='r')
  ax.add_patch(line)

  #right leg
  path = Path(keypoints[12:17:2],codes)
  line = patches.PathPatch(path, linewidth=2, facecolor='none', edgecolor='r')
  ax.add_patch(line)


In [ ]:
fig, ax = plt.subplots(1,figsize=(16,16))
ax.imshow(img)